<a href="https://colab.research.google.com/github/Ikhtishamanwar/ML-_Projects/blob/main/Cotton_diseases_Using_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name :Ikhtisham Anwar,
Cotton_diseases_Using_ResNet50


In [ ]:
# Extract the contents of the 'cotton_data (2).zip' file into the '/content' directory
import zipfile
zip_ref = zipfile.ZipFile('/content/cotton_data (2).zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
## Configure TensorFlow to use 50% of the GPU memory and allow memory growth, then start an interactive session
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
## Import necessary modules from TensorFlow and other libraries for building and processing a ResNet50 model

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
# Set the image size and define the paths for the training and validation datasets

IMAGE_SIZE = [224, 224]

train_path = '/content/cotton_data/train'
valid_path = '/content/cotton_data/test'


In [ ]:
# Load the ResNet50 model with pre-trained ImageNet weights, excluding the top layer, and set the input shape

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
# Set all layers in the ResNet50 model to be non-trainable
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
# Add a flattening layer to the output of the ResNet50 model
x = Flatten()(resnet.output)

In [ ]:
# Add a dense layer with 4 units and softmax activation for classification

prediction = Dense(units=4,activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
# Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Create image data generators for training (with augmentation) and testing (with rescaling)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Generate a training set from images in the '/content/cotton_data/train' directory with specified parameters

training_set = train_datagen.flow_from_directory('/content/cotton_data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [ ]:
# Generate a test set from images in the '/content/cotton_data/test' directory with specified parameters

test_set = test_datagen.flow_from_directory('/content/cotton_data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 18 images belonging to 4 classes.


In [ ]:
# Train the model on the training set, validate on the test set, with specified epochs and steps per epoch

r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
61/61 [==============================] - 41s 538ms/step - loss: 2.3545 - accuracy: 0.4254 - val_loss: 1.1731 - val_accuracy: 0.5556
Epoch 2/20
61/61 [==============================] - 28s 466ms/step - loss: 0.9722 - accuracy: 0.6007 - val_loss: 1.2564 - val_accuracy: 0.6111
Epoch 3/20
61/61 [==============================] - 29s 473ms/step - loss: 1.0069 - accuracy: 0.5894 - val_loss: 0.7777 - val_accuracy: 0.6111
Epoch 4/20
61/61 [==============================] - 29s 482ms/step - loss: 0.9345 - accuracy: 0.6274 - val_loss: 0.7200 - val_accuracy: 0.6667
Epoch 5/20
61/61 [==============================] - 29s 471ms/step - loss: 0.8707 - accuracy: 0.6525 - val_loss: 0.7445 - val_accuracy: 0.7222
Epoch 6/20
61/61 [==============================] - 29s 469ms/step - loss: 0.7638 - accuracy: 0.6786 - val_loss: 0.6906 - val_accuracy: 0.7778
Epoch 7/20
61/61 [==============================] - 29s 470ms/step - loss: 0.7745 - accuracy: 0.6961 - val_loss: 1.3862 - val_accuracy: 0.6111

In [ ]:
#The model was trained for 20 epochs, achieving a validation accuracy of 77.78%. The validation loss for the last epoch was 0.8662.